![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

Read in bank_marketing.csv as a pandas DataFrame.

In [206]:
import pandas as pd
import numpy as np

# Start coding here...

data = pd.read_csv("bank_marketing.csv") 


In [207]:
data.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

so the columns are
[
    'client_id', 'age', 'job', 
    'marital', 'education', 'credit_default',
    'housing', 'loan', 'contact', 
    'month', 'day','duration', 'campaign',
    'pdays', 'previous', 'poutcome', 
    'emp_var_rate', 'cons_price_idx','cons_conf_idx',
    'euribor3m', 'nr_employed', 'y'
]

In [208]:
data.info

<bound method DataFrame.info of        client_id  age          job  ... euribor3m nr_employed    y
0              0   56    housemaid  ...     4.857      5191.0   no
1              1   57     services  ...     4.857      5191.0   no
2              2   37     services  ...     4.857      5191.0   no
3              3   40       admin.  ...     4.857      5191.0   no
4              4   56     services  ...     4.857      5191.0   no
...          ...  ...          ...  ...       ...         ...  ...
41183      41183   73      retired  ...     1.028      4963.6  yes
41184      41184   46  blue-collar  ...     1.028      4963.6   no
41185      41185   56      retired  ...     1.028      4963.6   no
41186      41186   44   technician  ...     1.028      4963.6  yes
41187      41187   74      retired  ...     1.028      4963.6   no

[41188 rows x 22 columns]>

In [209]:
data.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


- Split the data into three DataFrames using information provided about the desired tables as your guide: one with information about the client, another containing campaign data, and a third to store information about economics at the time of the campaign.

In [210]:

client = data[["client_id", "age", "job", "marital", "education", "credit_default", "housing", "loan"]]

campaign = data[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]

economics = data[["client_id", "emp_var_rate", "cons_price_idx", 
                "euribor3m", "nr_employed"]]

Rename the column:
- "client_id" to "id" in **client** (leave as-is in the other subsets); 
- "duration" to "contact_duration", "previous" to "previous_campaign_contacts", "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts" in **campaign**; 
- and "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in **economics**.

In [211]:
client.rename(columns={"client_id":"id"}, inplace=True)

campaign.rename(columns={"duration":"contact_duration", "previous":"previous_campaign_contacts", "y":"campaign_outcome", "poutcome":"previous_outcome", "campaign":"number_contacts"}, inplace=True)

economics.rename(columns={"euribor3m":"euribor_three_months", "nr_employed":"number_employed"}, inplace=True)

In [212]:
economics.columns

Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')

Clean the "education" column:
- changing "." to "_" 
- and "unknown" to NumPy's null values.
- remove periods from the "job" column.

In [213]:
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [214]:
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)
client["job"] = client["job"].str.replace(".", "")
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


Convert "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0), along with the changing "nonexistent" to NumPy's null values in "previous_outcome".

In [215]:
campaign.head()

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,nonexistent,no
1,1,1,may,19,149,999,0,nonexistent,no
2,2,1,may,23,226,999,0,nonexistent,no
3,3,1,may,27,151,999,0,nonexistent,no
4,4,1,may,3,307,999,0,nonexistent,no


In [216]:
campaign["campaign_outcome"] = campaign["campaign_outcome"].replace("yes", 1).replace("no", 0).replace("failure", 0)

campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", 0).replace("failure", 0).replace("success", 1)

In [217]:
campaign.head()

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,0,0
1,1,1,may,19,149,999,0,0,0
2,2,1,may,23,226,999,0,0,0
3,3,1,may,27,151,999,0,0,0
4,4,1,may,3,307,999,0,0,0


- Add a column called campaign_id in campaign, where all rows have a value of 1.
- Create a datetime column called last_contact_date, in the format of "year-month-day", where the year is 2022,
-  and the month and day values are taken from the "month" and "day" columns.

In [218]:
campaign["campaign_id"] = 1

campaign["year"] = "2022"
campaign["last_contact_date"] = campaign["year"].astype(str) + "-" + campaign["month"].astype(str) + "-" + campaign["day"].astype(str) 



In [219]:
campaign.columns

Index(['client_id', 'number_contacts', 'month', 'day', 'contact_duration',
       'pdays', 'previous_campaign_contacts', 'previous_outcome',
       'campaign_outcome', 'campaign_id', 'year', 'last_contact_date'],
      dtype='object')

In [220]:
#order campaign columns:
campaign=campaign[['campaign_id', 'client_id', 'number_contacts', 'month', 'day', 'contact_duration',
       'pdays', 'previous_campaign_contacts', 'previous_outcome',
       'campaign_outcome', 'year', 'last_contact_date']]

In [221]:
campaign.columns

Index(['campaign_id', 'client_id', 'number_contacts', 'month', 'day',
       'contact_duration', 'pdays', 'previous_campaign_contacts',
       'previous_outcome', 'campaign_outcome', 'year', 'last_contact_date'],
      dtype='object')

In [222]:
#clean campaign columns
campaign.drop(columns=["month", "year", "day"], inplace=True)


In [223]:
campaign.head()

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,1,261,999,0,0,0,2022-may-13
1,1,1,1,149,999,0,0,0,2022-may-19
2,1,2,1,226,999,0,0,0,2022-may-23
3,1,3,1,151,999,0,0,0,2022-may-27
4,1,4,1,307,999,0,0,0,2022-may-3


- Save the three DataFrames to csv files without an index as 
- client.csv, 
- campaign.csv, 
- and economics.csv respectively.

In [224]:
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

- Create a variable called client_table, containing SQL code as a string to create a table called client using values from client.csv. 
 
- Create a variable called campaign_table, containing SQL code as a string to create a table called campaign using values from campaign.csv. 

- Create a variable called economics_table, containing SQL code as a string to create a table called economics using values from economics.csv.


- In client, campaign, and economic, ensure the final line copies the data from their respective csv files using the following code snippet:
\copy table_name from 'file_name.csv' DELIMITER ',' CSV HEADER

In [225]:
GRANT USAGE ON SCHEMA public TO PUBLIC;

""


In [226]:

client_table = """CREATE TABLE client 
(
    id SERIAL PRIMARY KEY, 
    age INTEGER, 
    job TEXT, 
    marital TEXT, 
    education TEXT, 
    credit_default BOOLEAN, 
    housing BOOLEAN, 
    loan BOOLEAN); 
    \copy client from 'client.csv' DELIMITER ',' CSV HEADER"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""


economics_table = """CREATE TABLE economics
(

     client_id SERIAL references client (id),
    emp_var_rate FLOAT NOT NULL,
    cons_price_idx FLOAT NOT NULL,
    euribor_three_months FLOAT NOT NULL,
    number_employed FLOAT NOT NULL
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

